# Indexing BM25 on Wikipedia Corpus

Source is Pyserini's own example:

https://github.com/castorini/pyserini/blob/master/docs/usage-index.md#building-a-bm25-index-embeddable-python-implementation

Steps omitted: extracting jsonl.gz first in my file system and afterwards with tar -xvf. (It has double compression, or I used the wrong decompression method) 

## 1. Indexing

This uses DefaultLuceneDocumentGenerator which already by default features stopword removal and stemming

- I have 14 threads, so I use half of them. More might be faster, but this seemed safe
- "--storePositions" "--storeDocvectors" "--storeRaw" are expensive options but we need the raw documents for reranking.

In [ ]:
# !python -m pyserini.index.lucene \
#   --collection JsonCollection \
#   --input data/data00/jiajie_jin/flashrag_indexes/wiki_dpr_100w \
#   --index indexes/wiki_dump \
#   --generator DefaultLuceneDocumentGenerator \
#   --threads 7 \
#   --storePositions --storeDocvectors --storeRaw

2025-11-12 13:43:02,659 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - Setting log level to INFO
2025-11-12 13:43:02,662 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) - ============ Loading Index Configuration ============
2025-11-12 13:43:02,662 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) - AbstractIndexer settings:
2025-11-12 13:43:02,670 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + DocumentCollection path: data/data00/jiajie_jin/flashrag_indexes/wiki_dpr_100w
2025-11-12 13:43:02,670 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + CollectionClass: JsonCollection
2025-11-12 13:43:02,671 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:215) -  + Index path: indexes/wiki_dump
2025-11-12 13:43:02,671 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:216) -  + Threads: 7
2025-11-12 13:43:02,671 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:217) -  + Optimize (merge segment

In [1]:
from pyserini.search.lucene import LuceneSearcher
import polars as pl
import json

query = 'Deaf Basketball'

searcher = LuceneSearcher('index/wiki_dump')
hits = searcher.search('Deaf Basketball',k=1000)

docs = [json.loads(hits[i].lucene_document.get('raw'))["contents"] for i in range(len(hits))]
docs[:2]

Nov 14, 2025 2:53:18 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


['"Deaf basketball"\nDeaf basketball Deaf basketball is basketball played by deaf people. Sign language is used to communicate whistle blows and communication between players. The game played by deaf people is organized with national and international associations including Deaf Basketball Australia, Deaf Basketball UK and United States of America Deaf Basketball. Deaf basketball has gained great visibility because of athlete like Lance Allred who played basketball with the National Basketball Association\'s (NBA) Cleveland Cavaliers. Allred is Hard of Hearing, with a 75-80% hearing loss wearing a hearing aid. He later on continued to play basketball professionally in the European basketball leagues. Another',
 '"Deaf basketball"\ndeaf basketball. DIBF encourages the growth and development of deaf basketball in all nationals of the world through an organized program of education and instruction. The Federation schedules and conducts all international contests and championships in deaf 

### Two variants (https://docs.langchain.com/oss/python/langchain/rag#rag-chains)


In [2]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from IPython.display import display, Markdown
from sentence_transformers import CrossEncoder


qwen = init_chat_model(model="ollama:qwen2.5:7B").bind(logprobs=True)
bm25 = LuceneSearcher('index/wiki_dump')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L6-v2')
query = "what is deaf basketball and what countries have this?"

def get_top_3_rerank(query: str) -> str:
    """Using the user query, this function retrieves the three most relevant models using rerank

    Args:
        query (str): the user query

    Returns:
        str: a string containing the top three documents.
    """    
    hits = bm25.search(query,k=1000)

    docs = [json.loads(hits[i].lucene_document.get('raw'))["contents"] for i in range(len(hits))]
    

    top_k = pl.DataFrame(cross_encoder.rank(query,docs,top_k=3,return_documents=True))
    
    return "\n".join(top_k.get_column('text').to_list())

agent = create_agent(
    model=qwen,
    tools=[get_top_3_rerank],
    system_prompt="You are a helpful assistant",
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]}
)
display(Markdown(response["messages"][-1].content))

Deaf basketball is a form of basketball specifically played by individuals who are deaf or hard of hearing. Communication on the court is facilitated through sign language for whistle blows and player coordination. Several countries actively participate in this sport, including:

1. **Australia**: Deaf Basketball Australia organizes competitions and events.
2. **United Kingdom (UK)**: Deaf Basketball UK manages local and national tournaments.
3. **United States**: United States of America Deaf Basketball participates in various leagues and championships.
4. **Finland**: Jussi Raisio, a notable deaf athlete, represented Finland at the International level.
5. **Sweden**: Sweden has been actively involved through organizations like Kjell Gunna.
6. **Lithuania**: Aleksas Jasiunas played a significant role in the formation of the Deaf International Basketball Federation (DIBF).
7. **China and Ukraine**: These countries are also part of the DIBF, contributing to the global deaf basketball community.

The Deaf International Basketball Federation (originally known as IDBA) was officially established to standardize rules and promote the sport internationally. This organization has played a crucial role in bringing together deaf athletes from various nations for competitions like the World Deaf Basketball Championships.

In [3]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Creates a prompt where the retrieval context is concatenated to the prompt

    Args:
        request (ModelRequest): The full list of messages. From this the function retrieves the last human message

    Returns:
        str: a new prompt with context(top 3 rerank) that should 
    """    
    """Inject context into state messages."""
    last_query = request.state["messages"][-1]

    retrieved_docs = get_top_3_rerank(last_query.content)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{retrieved_docs}"
    )

    return system_message


agent = create_agent(qwen, tools=[], middleware=[prompt_with_context])
response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]}
)
display(Markdown(response["messages"][-1].content))

Deaf basketball refers to the sport of basketball played primarily by individuals who are deaf or hard of hearing. Communication during games relies heavily on sign language to convey whistle blows and other important game information between players, officials, and coaches.

Several countries have active organizations and communities supporting deaf basketball:

1. **United States**: The United States has a strong presence in deaf basketball through organizations like the USA Deaf Basketball (USADB).
2. **Australia**: Deaf Basketball Australia organizes games and competitions for deaf players.
3. **United Kingdom**: Deaf Basketball UK operates to support deaf athletes and organize local and national competitions.
4. **Slovenia**: As mentioned, Slovenian player Miha Zupan has achieved high levels of professional play despite his hearing impairment.
5. **Other countries** likely have similar organizations or groups that promote deaf basketball at the national level.

These countries, along with others around the world, contribute to a vibrant and growing community of deaf athletes who compete in various tournaments and leagues dedicated to deaf sports. Additionally, there are international competitions like those organized by the Deaf International Basketball Federation (DIBF), which aligns with the broader Deaflympics movement to provide global visibility and opportunities for deaf athletes.

### Using huggingface (for white-box :()

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Code mostly taken directly from huggingface: https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def prompt_with_context_from_string(query: str) -> str:
    """Creates a prompt where the retrieval context is concatenated to the prompt

    Args:
        str(query): The human query as string.

    Returns:
        str: a new prompt with context(top 3 rerank) that should 
    """    
    """Inject context into state messages."""
    retrieved_docs = get_top_3_rerank(query)

    system_message = (
        "You are a helpful assistant. You answer in markdown. Use the following context in your response:"
        f"\n\n{retrieved_docs}"
    )

    return system_message

messages = [
    {"role": "system", "content": prompt_with_context_from_string(query)},
    {"role": "user", "content": query}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
display(Markdown(response))

Deaf basketball is a sport where deaf people play basketball using sign language instead of visual cues. It allows individuals who cannot hear to communicate through hand signals or vocalization. The sport has gained popularity due to the presence of a high-profile athlete named Lance Allred, who played basketball with the NBA Cleveland Cavaliers and won the league championship.

Several countries have supported the development and growth of deaf basketball:
1. Slovenia: Proving that deaf athletes can compete at the highest levels.
2. Italy: Has produced a number of talented players.
3. Canada: A country known for its strong deaf community and successful deaf basketball programs.
4. South Africa: Known for their diverse population and dedication to deaf sports.
5. Australia: Has seen a significant increase in deaf basketball participation over the past few years.
6. Switzerland: Offers a supportive environment for deaf athletes and has produced many notable players.
7. New Zealand: Known for its commitment to supporting deaf athletes and fostering deaf basketball culture globally.
These nations recognize the importance of deaf basketball and work towards developing a global movement for deaf athletes and their organizations.